In [1]:
# These are the ones we need for all the protocols
from Crypto.Util import number
import numpy as np
from phe import paillier
from Crypto.PublicKey import ElGamal
from ecpy.curves import Curve
from Crypto.Hash import SHA256
from Crypto.Util import number
import random
import math

# Review on privacy preserving data comparison protocols

[Jiang et al 2021](https://drive.google.com/file/d/1bn-PekGrzC-xot-B81oTEMRduEbq3asc/view?usp=sharing)

* Article reviews SMPC based comparison protocols
* Identify str. and weak. of various protocols
  * Adversary assumed as curious. 
  * With and without Trusted Third Party (TTP)
  * Comparison by category

* **Classification**

```
├── Secure Data Comparison Protocols
    ├── Comparison protocols for curious data owners
    │   ├── General Asymmetric Enc.
    │   └── Homomorphic Enc.
    │       ├── Sign Bit Decryption
    │       └── Difference Comparison
    └── Comparison protocols for curious cloud servers
        ├── With TTP
        │   ├── General Asymmetric Enc.
        │   └── Homomorphic Enc.
        └── Without TTP
            └── Homomorphic Enc.
```

* Asymmetric encryption based solutions always mix in interference items.
* **NOTE: THE JIANG PAPER CONTAINS LOTS OF ERRORS. IT IS STILL A WORTHY READ DUE TO THE CATEGORIZATION. CHECK ORIGINAL SOURCES AND BE CAREFUL**

**Title numbering below follows the title numbering in the paper**

## 3. Comparison protocols for curious data owners

General framework is as follows:

* Alice has pt $x$ and Bob has pt $y$ and want to perform some joint operation on these data
* Alice computes $c_A = E(x)$ and sends it to Bob.
* Bob computes $c_B = E(y)$
* Bob computes the encrypted magnitude relation using a comparison function $c = \texttt{comp}(c_A,c_B)$
* Alice computes $m = D(c)$ and gets the magnitude relation

### 3.2. Protocol with general public key encryption

This section of the paper is essentially [Yao's millionare problem](https://en.wikipedia.org/wiki/Yao%27s_Millionaires%27_problem)

In [2]:
# Below is from https://asecuritysite.com/encryption/mill2
# The paper is less clear than the example at asecuritysite

I = 38 # Alice's money
J = 39 # Bob's money

assert all([J, I]) > 0, "Values must be positive integers"

# Alice's parts is basic RSA key gen https://en.wikipedia.org/wiki/RSA_(cryptosystem)
p = number.getPrime(128)
q = number.getPrime(128)
N = p * q # Shared with Bob
PHI = (p-1) * (q-1) # Used for the inverse computation
e = number.getPrime(12) # Encryption key
d = number.inverse(e, PHI) # Decryption key

# Both
z = number.getPrime(64) # Alice and Bob agree on a new prime, should be L // 2 bits

# Bob's parts
U = number.getRandomNBitInteger(64)
C = pow(U, e, N)

# Bob computes 

m = C - J + 1

# print('\ne=',e,'d=',d,'\nN=',N,'z=',z)
# print('\nRandom Value U is:\t',U)
# print('C value is (U^e %N):\t',C)
# print("Bob shares this value:", m)

# Alice computes
Z=[]
for x in range(101):
    val = (pow(m + x, d, N)) % z
    if (x > (I - 1)):
        Z.append(val+1)
    else:
        Z.append(val)

# Alice sends Z to Bob, who computes
G = U % z

# print("\nG value is",G)
# print("Z values are:")
# for x in range(0,J):
#     print(Z[x],end=' ')

# print('\n\nCheck G(',G,') against the ',J,'th value (',Z[J-1],')')
print("\n I ≥ J") if (G==Z[J-1]) else print("\nJ > I")


J > I


Notes on the above protocol / scheme

* The random prime $z$ ensure that there is no identical elements in $\mathbf{Z}$
* No plaintext (pt) is transmitted
* Alice gets $m=C - J + 1$, but cannot compute $J$ because Alice has no knowledge of $U$

## 3.3. Protocols using HE

### 3.3.1 Bitwise methods

#### Ling Tzeng

The Lin Tzeng protocol is detailed [here](https://doi.org/10.1007%2F11496137_31). It reduces the GT problem to a set intersection problem, which is easy to compute in a privacy preserving way.

* Let $s=s_n s_{n-1} \ldots s_1 \in \{0,1\}^n$ be a binary string of length $n$.
* Denote 0-encoding of $s$ as the set $S^0_s=\{s_n s_{n-1} \ldots s_{i+1} 1 \mid s_i = 0, 1 \leq i \leq n\}$
* Denote 1-encoding of $s$ as the set $S^1_s=\{s_n s_{n-1} \ldots s_{i} \mid s_i = 1, 1 \leq i \leq n\}$

Given two values $(x,y)$, $x > y$ if and only if $S^1_x \cap S^0_y \neq \emptyset$, otherwise $x \leq y$. In their paper, Lin et al use bitwise encryption as well. Below is an example of this bitwise encryption. I did not fully implement it because it is too tedious. Also, there is a much faster way to do set comparison using the epione protocol, which I will show a bit further down.

To understand the bitwise encryption it is helpful to see an example. The encryption table for 50, which in binary is $\texttt{110010}$, is done as follows

|   | 1    | 1    | 0    | 0    | 1    | 0    |
|---|------|------|------|------|------|------|
| 0 | E(r) | E(r) | E(1) | E(1) | E(r) | E(1) |
| 1 | E(1) | E(1) | E(r) | E(r) | E(1) | E(r) |

* Note that `E(1) if b_i = 1 else E(r)` for the 1 row, and `E(0) if b_i = 0 else E(r)` in the 0 row. 
* The protocol relies on the fact that $E(1\times1\times\ldots) = E(1) \times E(1) \times \ldots $
    1. A person who wishes to compare numbers, simply multiplies the encrypted values in the table with the corresponding table locations
        * E.g., 100000 would be $(E(1 \times r \times 1 \times 1 \times r \times 1)$ which does not decrypt to 1. 
        * There is a set intersection if one of the ciphertext multiplications equals 1

In [3]:
# These are for the encoding part
def num_to_arr(n, length=None, print_flag=False):
    '''
    Takes as input a non negative integer
    Returns bin() representation in an array padded with leading zeros up to provided length
    '''
    arr = list(bin(n)[2:].zfill(length)) if length else list(bin(n)[2:])
    if print_flag: print('Binary stream of {}: '.format(n), bin(n)[2:].zfill(length)) if length else list(bin(n)[2:])
    return arr

def encoding(n, encoding_type, length=None, print_flag=False):
    '''
    Takes as input binary array
    Returns 0 or 1 encoding of array
    '''
    assert (encoding_type==0) or (encoding_type==1), "Type must be 0 or 1"
    assert n >= 0, 'Number must be non negative integer'
    arr = num_to_arr(n, length, print_flag)
    indices = np.where(np.array(arr) == str(encoding_type))[0]
    s = set()
    for i in indices:
        encoding = ''.join(arr[:i]+['1'])
        s.add(encoding)
    if print_flag: print('{}-encoding of {}: {}'.format(encoding_type, n, s))
    return(s)

def max_length(a, b):
    return max(len(num_to_arr(a)), len(num_to_arr(b)))

def yao_compare(a, b, length=None, print_flag=False):
    '''
    Takes two non zero integers as input
    Returns 
        * 1-encoding of a
        * 0-encoding of b
        * True if a <= b
    '''
    l = length if length else max_length(a, b)
    s_a_1 = encoding(a, encoding_type=1, length=l, print_flag=print_flag)
    s_b_0 = encoding(b, encoding_type=0, length=l, print_flag=print_flag)
    return s_a_1, s_b_0, s_a_1.intersection(s_b_0)==set()

In [5]:
a, b = 300, 200 # I use very small values because the encryption later is very tedious
_, _, less_or_equal = yao_compare(a, b, print_flag=True)
print('Result of the comparison: ', 'a <= b' if less_or_equal else 'a > b')

Binary stream of 300:  100101100
1-encoding of 300: {'1001', '1001011', '100101', '1'}
Binary stream of 200:  011001000
0-encoding of 200: {'01101', '0111', '1', '01100101', '011001001', '0110011'}
Result of the comparison:  a > b


**Note:** We must assume that Alice and Bob have communicated some length prior to their exchanges

In [7]:
alice_one_encoding, bob_zero_encoding = encoding(a, encoding_type=1, length=len(num_to_arr(a))+0, print_flag=False), encoding(b, encoding_type=0, length=len(num_to_arr(a))+1, print_flag=False)

In [4]:
%%time
# Alice generates encryption parameters
public_key, private_key = paillier.generate_paillier_keypair()
key = ElGamal.generate(256, ElGamal.Random.urandom) # this can take about 2 seconds

print(f'Public key components: \n(g={key.g}, p={key.p},y={key.y})')
print(f"Private key component: \n(x={key.x})")

# Alice sets parameters
g = int(key.g)
p = int(key.p)
x = int(key.x)
y = pow(g,x,p)

# Encrypt with a = g^k mod p; b = y^k * m mod p
# Decrypt with b / a^x mod p

Public key components: 
(g=107027176993253962048399027039980147559121710247963480063020185797615652236905, p=109819709321344122428449897431024610442361350555968641638837172957132041747639,y=50841775769456641206548990463374839380640682038589325317672966467565150937483)
Private key component: 
(x=95570804996811909230946955068140860338902194513027938968984848782573717445093)
CPU times: user 2.19 s, sys: 33.6 ms, total: 2.22 s
Wall time: 2.24 s


In [10]:
# These are for the encryption part
def ElGamalEncrypt(g, p, y, m):
    k = number.getRandomNBitInteger(p.bit_length() // 2 - 1) # to make sure that the random values are from G_q where p = 2q + 1
    a = pow(g, k, p)
    b = pow(y, k, p) * m % p
    return a, b

def table_creator(n):
    '''
    n is a binary string
    '''
    row_zeros = {}
    row_ones = {}
    counter = 0
    for i in n:
        if i == '1':
            row_ones[str(counter)] = ElGamalEncrypt(g,p,y,1)
            r = number.getRandomRange(2, 2 ** (p.bit_length() // 2 - 1))
            row_zeros[str(counter)] = ElGamalEncrypt(g,p,y,r)
            counter += 1
        else:
            row_zeros[str(counter)] = ElGamalEncrypt(g,p,y,1)
            r = number.getRandomRange(2, 2 ** (p.bit_length() // 2 -1))
            row_ones[str(counter)] = ElGamalEncrypt(g,p,y,r)
            counter += 1
    return row_zeros, row_ones

In [11]:
# This returns the encodings for each element in alices 1-encoding set
alice_to_bob = []
for i in alice_one_encoding:
    alice_to_bob.append(table_creator(i))
    
print('Alice sends to Bob the following tables (zeros first then ones):')
alice_to_bob

Alice sends to Bob the following tables (zeros first then ones):


[({'0': (69008217014735915427783864983428809178173839962654222796272592334751045965492,
    18609571965873580878488612143606587992977521772424637072037013011849167755767),
   '1': (72936577729180277451182315346789120221740165677131388500803423752214248292992,
    85113732599435533631345683226127330058918860213801673018185916955218872505621)},
  {'0': (1048487768014948844699806790008887055150519547173583788108938701478784587023,
    32969486044184613075916328412032056344426398902028926007372127542336873076467),
   '1': (71768786448810702137540084329700144472308397745412846958944251992203479738228,
    30628653186093156945520273331981337748056586916743117685132545270261910638703)}),
 ({'0': (260209436992711926725797682805896377290724387705630345288182748742277323094,
    13894790207823595887712449411187210048365993858832375685569384298610770759783)},
  {'0': (66585989557566998310954944181491891401289615448336700713010475395604393022577,
    55235049166810538052228134953860387533498587823

In [12]:
[x for x in alice_one_encoding] # just to make sure these are the same elements as in the set assuming Alice's number is 3

['11', '1']

In this simple example, we know that the element we will find is 11 (shared element between 1-encoding of 3 and 0-encoding of 2), so we use only that as example.

In [13]:
# We look at values for encoded '11' that Alice sends to Bob
# Note how Alice needs to send E(1) and E(r) for each i in '11', i.e., a total of four values where each value is a ciphertext pair under the El Gamal encryption
alice_to_bob[0]

({'0': (69008217014735915427783864983428809178173839962654222796272592334751045965492,
   18609571965873580878488612143606587992977521772424637072037013011849167755767),
  '1': (72936577729180277451182315346789120221740165677131388500803423752214248292992,
   85113732599435533631345683226127330058918860213801673018185916955218872505621)},
 {'0': (1048487768014948844699806790008887055150519547173583788108938701478784587023,
   32969486044184613075916328412032056344426398902028926007372127542336873076467),
  '1': (71768786448810702137540084329700144472308397745412846958944251992203479738228,
   30628653186093156945520273331981337748056586916743117685132545270261910638703)})

In [14]:
# We know where E(1) is and we use that to test if above has worked by decrypted the E(1) locations
pow(pow(alice_to_bob[0][1]['0'][0], x, p), -1, p) * alice_to_bob[0][1]['0'][1] % p,\
pow(pow(alice_to_bob[0][1]['1'][0], x, p), -1, p) * alice_to_bob[0][1]['1'][1] % p

(1, 1)

In [15]:
# Just to ensure that the other values are random and not 1
pow(pow(alice_to_bob[0][0]['0'][0], x, p), -1, p) * alice_to_bob[0][0]['0'][1] % p,\
pow(pow(alice_to_bob[0][0]['1'][0], x, p), -1, p) * alice_to_bob[0][0]['1'][1] % p

(67765509584254404343994876521810025248,
 158361609190254901501992239896279692003)

In [16]:
# Bob now takes Alices table values based on his own indexes and multiplies the values together.
# Bob has to pick values for '11' since that is his zero encoding. Again we only care about the '11' element in the set.
alice_to_bob[0][1]['0'], alice_to_bob[0][1]['1']

((1048487768014948844699806790008887055150519547173583788108938701478784587023,
  32969486044184613075916328412032056344426398902028926007372127542336873076467),
 (71768786448810702137540084329700144472308397745412846958944251992203479738228,
  30628653186093156945520273331981337748056586916743117685132545270261910638703))

In [18]:
# Bob essentially multiplies the El Gamal (a,b) b values corresponding to the table index he has for '11'
# Note that the El Gamal b value is not Bob's number b but the second element of the ciphertext pair
b_multiples = alice_to_bob[0][1]['0'][1] * alice_to_bob[0][1]['1'][1] % p

# Alice can now check if the multiplications equal 1
# Note that we have not yet scalarized these values
a_multiples = pow(pow(alice_to_bob[0][1]['0'][0], x, p), -1, p) * pow(pow(alice_to_bob[0][1]['1'][0], x, p), -1, p)
b_multiples * a_multiples % p

1

**Scalaring**
* We can map $E(m)$ to a random value with a scalarizing function $S: E(m) \longrightarrow E(m^k)$
* When $m=1$ the scalarized value still decrypts without a problem
* When $m \ne 1$ then the scalarized value will no longer decrypt to $r_i$
* Scalaring hides any value from being exposed to Alice.
    1. E.g., if Alice has value '1001' and bob has value '1000' then Alice would get $E(1 \cdot 1 \cdot 1 \cdot r_3)$ and since Alice knows the value of $r_3$ she now knows Bob's number

In [19]:
# Alice can tell what element they had in common using the above.
# Bob must hide the values using scalaring
scaling_value = number.getRandomRange(2, 2 ** p.bit_length() // 2 - 1)
b_multiples_scaled = pow(alice_to_bob[0][1]['0'][1], scaling_value, p) * pow(alice_to_bob[0][1]['1'][1], scaling_value, p) % p
a_multiplies_scaled = pow(alice_to_bob[0][1]['0'][0] * alice_to_bob[0][1]['1'][0], scaling_value, p)

In [20]:
# Alice now tests that the product of the scalarized values are equal to 1
print('Alice decrypts the scalarized product: ', b_multiples_scaled * pow(pow(a_multiplies_scaled, x, p), -1, p) % p)

Alice decrypts the scalarized product:  1


#### Lin Tzeng with Epione comparison instead of bitwise (THIS IS MY OWN COMBINATION AND HAS NOT BEEN REVIEWED OR CHECKED BY OTHERS)

The bitwise approach above seems to be a mess to implement.
A much easier way is to use the epione protocol for Private Set Intersection

In [22]:
# We use a hashing function to uniquely map set elements in the encodings to integer values we can use for set intersection proofs
def sha256(m):
    '''
    Takes input m
    Ouputs SHA256(m)
    '''
    m = m.encode('utf-8')
    H = SHA256.new()
    H.update(m) 
    return H.hexdigest()

In [23]:
# Alice and Bob agree on a curve and a generator point
curve = Curve.get_curve('Curve25519')
G = curve.generator
order = curve.order

In [27]:
# Alice gets her encoding and computes the encrypted set for each
# Note how we can use a lot larger numbers now with ease
a = 54000000001
b = 54000000000
s_1_a, s_0_b, less_or_equal = yao_compare(a, b, print_flag=False) # the (0,1)-encoding

# Create sets of hash digests from the encoding elements
alice_values = [int(sha256(x), base=16) for x in s_1_a]
bob_values = [int(sha256(x), base=16) for x in s_0_b]

# Alice hides her true values using a random point and sends to Bob. Note that finding r is assumed hard.
r = number.getRandomNBitInteger(256) # Alice random
raAi = [r * alice_values[i] * G for i in range(len(alice_values))]
inv_r = pow(r, -1, order)

# Bob multiplies the values he gets from Alice with random and sends back to Alice
s = number.getRandomNBitInteger(256) # Bob random
sBi = [s * bob_values[i] * G for i in range(len(bob_values))] # Bob multiplies his values with a random s
sraAi = [s * raAi[i] for i in range(len(raAi))] # Bob multiplies Alices values with random s
random.shuffle(sraAi) # Bob shuffles the elements so that order is not preserved

# Alice takes the shuffled values and undoes the multiplication with r
sAi = [inv_r * sraAi[i] for i in range(len(sraAi))]

# Alice now checks for a set intersection
print('a > b') if len(np.intersect1d([sBi[i].x for i in range(len(sBi))], [sAi[i].x for i in range(len(sAi))])) else print('a ≤ b')

a > b


**Note that we can use any encryption scheme for the private set intersection that relies on the [Diffie Hellman assumptions](https://en.wikipedia.org/wiki/Computational_Diffie%E2%80%93Hellman_assumption)**. All we need is that reversing the multiplication with random values to be computationally intractable. I used Elliptic Curves because I have already done the RSA method elsewhere. I have copy pasted the RSA method below. The problem is that it does not work well with very large numbers.

In [ ]:
## RSA based Yao

# p = number.getPrime(128) # bit size prime # Only Bob knows
# q = number.getPrime(128) # bit size prime # Only Bob knows
# N = p * q
# assert p!=q, 'p and q are the same'

# phi = (p-1) * (q-1) # Only Bob knows
# e = number.getPrime(32) # bit size prime
# d = number.inverse(e, phi)

# a_value = 4
# b_value = 5

# U = number.getRandomNBitInteger(28) # random number generated by Alice, not shared with Bob
# m = pow(U, e, N) - a_value # Alice does this.
# z = number.getPrime(16) # Bit size prime. Known to both.

# # Z = []
# # for x in range(0,11):
# #     Z.append(pow(m + x, d, N) % z)
    
# Z = pow(m + b_value, d, N) % z

# print('Public key is:\nN: %d\ne: %d' %(N,e))
# print('\nPrivate key known to Bob is:\nN: %d\nd: %d' %(N,d))
# print('\nAlice picks random U:', U)
# print('Alice sends to Bob m: {}'.format(m))
# print('Both agree on prime z: {}'.format(z))
# print('\nAlice computes U mod z:', U % z)
# print('Bob sends to Alice (m + bob_value)^b % N % z: {}'.format(Z))
# print('Alice checks equality:', Z == U % z)

#### Hu et al

Original paper found [here](https://doi.org/10.1109/TIP.2016.2568460)

* Another encoding of bitstream based method
* Computes the magnitude relationship of two values by bit in binary
* Expresses values as $n$-bit bitstreams
    1. Alice's values are $x=x_n x_{n-1} \ldots x_{1}$
    2. Bob's values are $y=y_n y_{n-1} \ldots y_{1}$
* Alice does $c=c_n c_{n-1} \ldots c_{1}$, where $c_i = E(x_i), 1 \leq i \leq n$
* Alice sends $c$ and the public key to Bob
* Bob computes $E(1) \text{ and } E(-1)$
* Bob then compares the two streams as follows
    1. If $i = n$
        * $a_i = x_i \times (1-y_i)$
        * $b_i = y_i \times (1-x_i)$
    2. else 
        * $a_i = (1-b_{i+1}) \times (a_{i+1} + (1-a_{i+1})) \times x_i \times (1-y_i)$
        * $b_i = (1-a_{i+1}) \times (b_{i+1} + (1-b_{i+1})) \times y_i \times (1-x_i)$
    3. If $a_1 = 1$ then we know that $x \gt y$
* Uses LWE with SIMD support for efficiency

Below I only show how the method works. I will not encrypt but show it using plaintext that can be encrypted with similar results.

In [103]:
# We get two values to compare
A = number.getRandomNBitInteger(8)
B = number.getRandomNBitInteger(8)

In [102]:
# binary reprentation of the two values
A, B = bin(A)[2:], bin(B)[2:]

In [28]:
def HuComparison(A,B):
    assert (A >= 0) == (B >= 0), 'Use non negative integers'
    A, B = bin(A)[2:], bin(B)[2:]  #remove the 0b prefix
    
    if len(A) < len(B):
        A = A.zfill(len(B))  
    else:
        B = B.zfill(len(A))
    
    a = []
    b = []    
    
    a.append(int(A[0]) * (1 - int(B[0]))) # Only multiplication and addition used ie SIMD HE friendly
    b.append(int(B[0]) * (1 - int(A[0]))) # Only multiplication and addition used ie SIMD HE friendly
    
    for i in range(1, len(A)):        
        a_i = (1 - b[i-1]) * (a[i-1] + (1 - a[i-1]) * int(A[i]) * (1 - int(B[i]))) # Only multiplication and addition used ie SIMD HE friendly
        b_i = (1 - a[i-1]) * (b[i-1] + (1 - b[i-1]) * int(B[i]) * (1 - int(A[i]))) # Only multiplication and addition used ie SIMD HE friendly
        a.append(a_i)
        b.append(b_i)
    return a[::-1][0] # We only care about LSB of original bitsream

In [29]:
HuComparison(1024,32), HuComparison(32,1024), HuComparison(32,32), HuComparison(32,32), HuComparison(1,0), HuComparison(0,0)

(1, 0, 0, 0, 1, 0)

### 3.2. Difference comparison method

These protocols do not work on bitstreams but instead work on entire words

#### Kumar & Gupta

Full paper [here](https://arxiv.org/pdf/1310.8063.pdf)

**Protocol A**

* A very simple solution to the Yao's millionare problem using HE
* Alice generates $(pk,sk)$
* Alice has a number $a$ and computes $E(a)$ and sends this to Bob
* Bob has a number $b$ and computes $E(b)$ 
* Bob computes $V = E(r) \oplus E(a-b)$ where $r \longleftarrow \texttt{rand()}$. 
    * Note that $E(a)-E(b)=E(a-b)$.
* Alice decrypts $V$ to obtain $(a-b)\oplus r$
* Alice sends the most significant bit to Bob
* Bob takes the most significant bit sent by Alice and compares it with the MSB of $r$
* Bob shares the results with Alice

**Protocol B**

* Alice and Bob agree on an order preserving function $F: A \mapsto B$  (where $|A| \ll |B|$) s.t. $a < b \iff F(a) < F(b), \forall a,b \in A$
    * E.g., $f(x)=2 x$ is order preserving
* Trent does not know $F$
* Alice sends $F(a)$ to Trent
* Bob sends $F(b)$ to Trent
* Trent shares `F(a) < F(B)`

The paper defines the following two constraints of an order preserving function

1. $(f_i(1) - f_i(0)) \gt \sum_{j=1}^{i-1}(f_j(1) - f_j(0))$
2. $f_i(1) \gt f_i(0), \forall i$ 

I.e., the difference between two $i$th position values must be larger than the sum of all previous differences and the function must map $1$ to a greater value than $0$ for all positions. An example follows.

| f_i | f_i(0) | f_i(1) | f_i(1)-f_i(0) | \sum_{j=1}^{i}(f_j(1) - f_j(0)) |
|-----|--------|--------|---------------|---------------------------------|
| f_1 | 2      | 5      | 3             | 3                               |
| f_2 | 1      | 7      | 6             | 9                               |
| f_3 | 7      | 17     | 10            | 19                              |
| f_4 | 2      | 22     | 20            | 39                              |

In [30]:
# This is just an example of an order preserving function. It is not the same one as shown in the paper.
# It would have been smarter to start with the f_i(0) and f_i(1) values as shown in the paper but whatever

def sequence_generator(n, k=10):
    '''
    Outputs an array with sum difference values
    '''
    sequence = []
    for i in range(n):
        sequence.append(sum(sequence) + number.getRandomInteger(k))
    return sequence

def order_preserving_function(n, k=10):
    """
    Outputs sequence of sums, and the corresponding lookuptable values
    """
    s = sequence_generator(n, k)
    f_i_1 = []
    f_i_0 = []
    for i in range(n):
        r = 0
        while r == 0:
            r = number.getRandomInteger(k)
        f_i_1.append(s[i]+r)
        f_i_0.append(f_i_1[i] - s[i])
    return s, f_i_0, f_i_1

def order_preserving_function_bin_value(n, opf=None, k=10):
    '''
    Outputs the order preserving function output of an integer array
    '''
    if opf:
        f_i_0, f_i_1 = opf[0], opf[1]
    else:
        _, f_i_0, f_i_1 = order_preserving_function(math.ceil(math.log2(max(n))), k)
    
    result = []
    for i in n:
        temp = []
        x = bin(i)[2:][::-1]   
        for i in range(len(x)):
            if int(x[i]) == 0:
                temp.append(f_i_0[i])
            else:
                temp.append(f_i_1[i])
        result.append(sum(temp))
    return result

In [32]:
order_preserving_function_bin_value([10,2,2,0,8])

[3923, 755, 755, 237, 3421]

**Protocol C**
 
The paper introduces a way of using 1 out of 2 oblivious transfer to construct a random order preserving function (OPF) in such a way that neither Alice nor Bob know the true nature of the OPF. All they know is that it preservers the order at a certain point. I will skip this protocol because I look closer at oblivious transfer based techniques at another point in time.

### Comparison of protocols

* Transmission of public key is not considered
* Assume that $n$ numbers are compared

**Performance comparison**

| Protocol | Comp. complexity | Data amount   | Tx time   |
|----------|------------------|---------------|-----------|
| Yao      | $O(n N)$         | $(1+N)C_n^2$  | $3C_n^2$  |
| Lin      | $O(n k)$         | $4kC_n^2$     | $2C_n^2$  |
| Hu       | $O(n)$           | $2C_n^2$      | $2C_n^2$  |
| Kumar    | $O(n)$           | $2C_n^2$      | $2C_n^2$  |

**Security comparison**

| Protocol | Data leak to A | Data leak to B | Probability of A deriving pt | Probability of B deriving pt |
|----------|----------------|----------------|------------------------------|------------------------------|
| Yao      | None           | None           | $1/N$                        | $1/N$                        |
| Lin      | None           | None           | $1/N$                        | $1/N$                        |
| Hu       | None           | None           | None                         | None                         |
| Kumar    | $r(x-y)$       | None           | None                         | None                         |

## 4. Comparison protocols using a TTP

* Cases where TTP are involved
* Here the data owner does not take part in the comparison, but instead only provides data
* Data owner sends ciphertext to other entity and private key to TTP
* Everyone is assumed to be curious about the data owner's data
* It is up to the entity to transform the ciphertext in such a way that the TTP does not access the underlying data
* The TTP converts data into comparable data objects and sorts the data
* TTP shares the sorting results with the entity

A general framework is 

1. Data owner generates $(pk,sk)$ and sends $sk$ to Trent
2. Encryption is $E_{pk}(\bullet)$
3. Decryption is $D_{sk}(\bullet)$
4. The ciphertext of $E_{pk}(x_i)$ is $c_i$.
5. The TTP can sort $D_{sk}(c_i), 1\leq i \leq \vert c \vert$ but cannot learn any singe $x_i$

### 4.2. Protocols with asymmetric encryption

Kaghazgaran et al does two party comparison as detailed [here](https://doi.org/10.1109/WICT.2011.6141405)

* Alice generates $(pk,sk)$ and shares $sk$ with Trent and $pk$ with Bob
* Alice computes $E_{pk}(x_i),1 \leq i \leq n$
* Bob picks a random integer $r, 1\lt r \lt L$ and computes the values of $c_i = E_{pk}(x_i)-r$
* Trent computes $m_{i,j} = D_{sk}(c_i + j), 1 \leq j \leq L$
* Trent picks a random value $s \longleftarrow \texttt{rand()}$ and computes $M_{i,j} = m_{i,j} + s$
* Bob can now look at the $l$-th number of the array sent by Trent and sort the data.

The above seems to be a combination of several of the protocols above so I will not implement it here.

#### 4.3. Protocols with HE

Kaghazgaran et al also propose a more efficient version of their protocol detailed in 4.2. by using HE for the addition of random values.

## 5. Comparison protocols without a TTP

Similar to 4. above but Alice never sends $sk$ to Trent. 

* Instead of Trent, a compute resources gets an incomplete decryption algorithm $D'(\bullet)$ and a sorting algorithm $F(\bullet)$ specific to the encryption used and Alice's $sk$

### 5.2. Protocols with HE

* Hsu ([source](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6216412&casa_token=c1c3bFzIXtYAAAAA:dd9Bbc2GpJCk12Zza7_X5MoQrp0VunFciprf5GYXPBCWKR4T1ftmXnalRA24iN2NJIUxbdsqUH81&tag=1)) and further work by Bellafqira ([source](https://arxiv.org/pdf/1704.00457.pdf)) proposes a protocol based on threshold. These methods eliminate random values and are deemed less secure by the review authors.
* Jiang has another method that is more secure but I will have to look into it in depth to see how it works. **Edit**: It is the same as the random value scalaring and then just substract.

# My personal notes on the review paper and the procols shown

* In essence, you either encrypt the entire msg or bitwise. 
* Bitwise ones require encoding methods and 'tricks' to convert a number to 
    1. something where a specific bit can be compared
    2. sets which can then be compared
    3. arrays with values where some ith value is compared
* msg based ones often rely on multiplications with random values, their inverses, and order shuffling
* msg based ones above rely on the DH assumptions

I did not do an analysis, but I believe that the best one is a combination of the Lin encoding where hashed set elements are used for the epione private intersection cardinality protocol. The only problem I see with that one is that it requires the parties to agree on some msg length. Note that you can `zfill` both parties binary representations to some arbitrary chosen length and in so doing hide the bit length of the largest integer. It would be nice to figure out a way to implement the Lin encoding s.t. it does not care about the max binary length of the values that are compared.